# Aula de algoritmos de redução de dimensionalidade

Nesta aula vamos colocar às mãos no código para ter uma visão prática do uso de algoritmos de aprendizado de máquina para extrair um entendimento mais profundo de uma base de dados. Desta vez, vamos implementar algoritmos de redução de dimensionalidade para processar a base de dados como um todo. Além disso, vamos implementar o conceito aprendido sobre as diferentes etapas no desenvolvimento de uma aplicação com aprendizado de máquina.

Neste exemplo, vamos usar a base de dados que sumariza 82 características de 21.263 materiais supercondutores diferentes. Esses dados foram compilados de artigos científicos pelo Superconducting Material Database maintained by Japan’s National Institute for Materials Science (NIMS), e são disponíveis Materials Data Repository (MDR).

A tabela que utilizaremos é uma versão compilada e disponível em: https://www.kaggle.com/datasets/munumbutt/superconductor-dataset?select=train.csv

## Importando bibliotecas e dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

## Preparando o ambiente

In [ ]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/tiagofiorini/MLinPhysics.git
    import os as os
    os.chdir('./MLinPhysics')

## Carregando os dados

In [ ]:
dados = pd.read_csv('Superconductivity.csv')
X = dados
dados

O nome atribuído às 82 características dos materiais são:

In [ ]:
dados.columns

In [ ]:
import seaborn as sns
matriz_corr = dados.corr().round(2)
sns.heatmap(matriz_corr, cmap = 'RdBu', vmin=-1, vmax=1);

## Pré processamento

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

## Modelo 1: Fatorização

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X)
expl = pca.explained_variance_ratio_
svalues = pca.singular_values_
x = pca.transform(X)

In [ ]:
plt.hist2d(x[:,0],x[:,1],bins=50,  norm=LogNorm());

### Otimização 1: Número de fatores

In [ ]:
explainability = pca.explained_variance_ratio_.cumsum()
factors = np.arange(1,X.shape[1]+1,1)
plt.scatter(factors,explainability)
plt.hlines(0.9,0,20,'r')
plt.xlabel('Número de componentes')
plt.ylabel('Explicabilidade dos dados')

p = factors[explainability<0.9].max()+1

if p<2:
    p=2
    
print('90%% dos dados são explicados com as ' + str(p) + ' componentes.')

pca = PCA(n_components=p)
pca.fit(X)
x = pca.transform(X)

#x = x[:,0:p]
#x.shape

## Modelo 2: Clusterização

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5).fit(x)
categorias = kmeans.labels_

plt.scatter(x[:,0], x[:,1], c=categorias)
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2');

### Otimização 2: Número de clusters

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

Ns = np.asarray([2, 3, 4, 5, 6, 7, 8, 9])
silhueta_scores = np.zeros(len(Ns))
pontos_bad = np.zeros(len(Ns))

for i,N in enumerate(Ns):
    kmeans = KMeans(n_clusters=N).fit(x)
    categorias = kmeans.labels_
    silhueta_scores[i] = silhouette_score(x, categorias)
    sample_silhouette_values = silhouette_samples(x, categorias)
    mask = ( sample_silhouette_values < 0 )
    pontos_bad[i] = (100 * mask.sum() / len(mask)).round(2)
    
        
plt.plot(Ns,silhueta_scores);
plt.xlabel('Clusters')
plt.ylabel('Fator de silhueta')

plt.figure()
plt.plot(Ns,pontos_bad)
plt.xlabel('Clusters')
plt.ylabel('Pontos mal atribuídos')

plt.figure()
plt.plot(Ns,silhueta_scores/pontos_bad)
plt.xlabel('Clusters')
plt.ylabel('Métrica de performance')

opt_n_clusters = Ns[(silhueta_scores/pontos_bad).argmax()]

In [ ]:
kmeans = KMeans(n_clusters=opt_n_clusters).fit(x)
#kmeans = KMeans(n_clusters=2).fit(x)
categorias = kmeans.labels_

plt.figure()
plt.scatter(x[:,0], x[:,1], c=categorias)
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2');

print(str(opt_n_clusters) + ' é o número de clusters que apresenta o melhor fator de silhueta.')

## Pós processamento

In [ ]:
centros = kmeans.cluster_centers_

#centros = np.pad(centros, ((0,0),(0,X.shape[1]-p)), mode='constant')

centros_back = pca.inverse_transform(centros)
centros_back = scaler.inverse_transform(centros_back)

ordem = centros_back[:,-1].argsort()
centros_back = centros_back[ordem,:]

dados_provisorio = pd.DataFrame(data=centros_back, columns=dados.columns)
dados_provisorio

In [ ]:
import matplotlib.colors as mcolors

colors = [(1,0,0,c) for c in np.linspace(0,1,100)]
cmapred  = mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(0,1,0,c) for c in np.linspace(0,1,100)]
cmapgreen  = mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(0,0,1,c) for c in np.linspace(0,1,100)]
cmapblue =  mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(1,0,1,c) for c in np.linspace(0,1,100)]
cmappurple =  mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(0,1,1,c) for c in np.linspace(0,1,100)]
cmapcian =  mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(1,1,0,c) for c in np.linspace(0,1,100)]
cmapyel =  mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(0.5,0.5,0.1,c) for c in np.linspace(0,1,100)]
cmapcor1 =  mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)
colors = [(0.1,0.4,0.5,c) for c in np.linspace(0,1,100)]
cmapcor2 =  mcolors.LinearSegmentedColormap.from_list('mymap', colors, N=5)


Xpart = np.array([dados['mean_ElectronAffinity'], dados['critical_temp']]).T

colormaps = [cmapred, cmapgreen, cmapblue, cmappurple, cmapcian, cmapyel, cmapcor1, cmapcor2]
for i in range(categorias.max()+1):
    mask = ( categorias == i )
    xpart = Xpart[mask,:]
    plt.hist2d(xpart[:,0],xpart[:,1],bins=[np.linspace(0,300,50),np.linspace(0,200,50)],cmap=colormaps[i])
plt.xlabel('Afinidade eletrônica')
plt.ylabel('Temperatura crítica');

In [ ]:
u = ['mean_ElectronAffinity', 'gmean_Valence', 'gmean_atomic_mass']

for i in range(len(u)):
    plt.figure()
    plt.plot(dados_provisorio['critical_temp'], dados_provisorio[u[i]],label=u[i])
    plt.ylabel(u[i])
    plt.xlabel('Temperatura Crítica')
    